In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

from bs4 import BeautifulSoup
import requests
import time

In [2]:
df_movies = pd.read_csv('movies.csv') 

In [3]:
df_ratings = pd.read_csv('ratings.csv')

In [4]:
df_links = pd.read_csv('links.csv', dtype = object)
df_links.fillna(value=0, inplace=True)

In [5]:
df_tags = pd.read_csv('tags.csv')

In [ ]:
df_movies

In [ ]:
df_ratings

In [ ]:
df_links

In [ ]:
df_tags

In [6]:
header = {"accept-language": "es-ES"}

In [ ]:
def cargarSinopsis():
    for i in df_links["tmdbId"]:
        link = "https://www.themoviedb.org/movie/" + i
        r = requests.get(link, headers = header)
        bs = BeautifulSoup(r.content, "html.parser")
        print(bs.p.get_text())
        

In [ ]:
print(cargarSinopsis())

In [ ]:
for i in df_links["tmdbId"]:
    print(i) 
    link = "https://www.themoviedb.org/movie/" + str(i)
    r = requests.get(link, headers = header)
    bs = BeautifulSoup(r.content, "html.parser")
    try:
        sinopsis = bs.p.get_text()
        print(sinopsis)
    except:
        print("No se encontro sinopsis para el tmdbId: ",i)
        continue
    time.sleep(1)


In [ ]:
results = []
for i in df_links["tmdbId"]:
    link = "https://www.themoviedb.org/movie/" + str(i)
    r = requests.get(link, headers = header)
    bs = BeautifulSoup(r.content, "html.parser")
    try:
        sinopsis = bs.p.get_text()
        results.append([i,sinopsis])
    except:
        results.append([i,"No se encontro sinopsis"])
        continue
    time.sleep(1)
    
df = pd.DataFrame(results,columns=['tmdbId','sinopsis'])
df.to_csv('sinopsis.csv', index=False)

In [7]:
results = []
for i, tmdbId in enumerate(df_links["tmdbId"]):
    link = "https://www.themoviedb.org/movie/" + str(tmdbId)
    r = requests.get(link, headers = header)
    bs = BeautifulSoup(r.content, "html.parser")
    try:
        sinopsis = bs.p.get_text()
        results.append([tmdbId,sinopsis])
    except:
        results.append([tmdbId,"No se encontro sinopsis"])
        continue
    time.sleep(1)
    print(f"{i+1}/{len(df_links)} completed")

df = pd.DataFrame(results,columns=['tmdbId','sinopsis'])
df.to_csv('sinopsis.csv', index=False)

1/9742 completed
2/9742 completed
3/9742 completed
4/9742 completed
5/9742 completed
6/9742 completed
7/9742 completed
8/9742 completed
9/9742 completed
10/9742 completed
11/9742 completed
12/9742 completed
13/9742 completed
14/9742 completed
15/9742 completed
16/9742 completed
17/9742 completed
18/9742 completed
19/9742 completed
20/9742 completed
21/9742 completed
22/9742 completed
23/9742 completed
24/9742 completed
25/9742 completed
26/9742 completed
27/9742 completed
28/9742 completed
29/9742 completed
30/9742 completed
31/9742 completed
32/9742 completed
33/9742 completed
34/9742 completed
35/9742 completed
36/9742 completed
37/9742 completed
38/9742 completed
39/9742 completed
40/9742 completed
41/9742 completed
42/9742 completed
43/9742 completed
44/9742 completed
45/9742 completed
46/9742 completed
47/9742 completed
48/9742 completed
49/9742 completed
50/9742 completed
51/9742 completed
52/9742 completed
53/9742 completed
54/9742 completed
55/9742 completed
56/9742 completed
5

438/9742 completed
439/9742 completed
440/9742 completed
441/9742 completed
442/9742 completed
443/9742 completed
444/9742 completed
445/9742 completed
446/9742 completed
447/9742 completed
448/9742 completed
449/9742 completed
450/9742 completed
451/9742 completed
452/9742 completed
453/9742 completed
454/9742 completed
455/9742 completed
456/9742 completed
457/9742 completed
458/9742 completed
459/9742 completed
460/9742 completed
461/9742 completed
462/9742 completed
463/9742 completed
464/9742 completed
465/9742 completed
466/9742 completed
467/9742 completed
468/9742 completed
469/9742 completed
470/9742 completed
471/9742 completed
472/9742 completed
473/9742 completed
474/9742 completed
475/9742 completed
476/9742 completed
477/9742 completed
478/9742 completed
479/9742 completed
480/9742 completed
481/9742 completed
482/9742 completed
483/9742 completed
484/9742 completed
485/9742 completed
486/9742 completed
487/9742 completed
488/9742 completed
489/9742 completed
490/9742 com

870/9742 completed
871/9742 completed
872/9742 completed
873/9742 completed
874/9742 completed
875/9742 completed
876/9742 completed
877/9742 completed
878/9742 completed
879/9742 completed
880/9742 completed
881/9742 completed
882/9742 completed
883/9742 completed
884/9742 completed
885/9742 completed
886/9742 completed
887/9742 completed
888/9742 completed
889/9742 completed
890/9742 completed
891/9742 completed
892/9742 completed
893/9742 completed
894/9742 completed
895/9742 completed
896/9742 completed
897/9742 completed
898/9742 completed
899/9742 completed
900/9742 completed
901/9742 completed
902/9742 completed
903/9742 completed
904/9742 completed
905/9742 completed
906/9742 completed
907/9742 completed
908/9742 completed
909/9742 completed
910/9742 completed
911/9742 completed
912/9742 completed
913/9742 completed
914/9742 completed
915/9742 completed
916/9742 completed
917/9742 completed
918/9742 completed
919/9742 completed
920/9742 completed
921/9742 completed
922/9742 com

1287/9742 completed
1288/9742 completed
1289/9742 completed
1290/9742 completed
1291/9742 completed
1292/9742 completed
1293/9742 completed
1294/9742 completed
1295/9742 completed
1296/9742 completed
1297/9742 completed
1298/9742 completed
1299/9742 completed
1300/9742 completed
1301/9742 completed
1302/9742 completed
1303/9742 completed
1304/9742 completed
1305/9742 completed
1306/9742 completed
1307/9742 completed
1308/9742 completed
1309/9742 completed
1310/9742 completed
1311/9742 completed
1312/9742 completed
1313/9742 completed
1314/9742 completed
1315/9742 completed
1316/9742 completed
1317/9742 completed
1318/9742 completed
1319/9742 completed
1320/9742 completed
1321/9742 completed
1322/9742 completed
1323/9742 completed
1324/9742 completed
1325/9742 completed
1326/9742 completed
1327/9742 completed
1328/9742 completed
1329/9742 completed
1330/9742 completed
1331/9742 completed
1332/9742 completed
1333/9742 completed
1334/9742 completed
1335/9742 completed
1336/9742 completed


1697/9742 completed
1698/9742 completed
1699/9742 completed
1700/9742 completed
1701/9742 completed
1702/9742 completed
1703/9742 completed
1704/9742 completed
1705/9742 completed
1706/9742 completed
1707/9742 completed
1708/9742 completed
1709/9742 completed
1710/9742 completed
1711/9742 completed
1712/9742 completed
1713/9742 completed
1714/9742 completed
1715/9742 completed
1716/9742 completed
1717/9742 completed
1718/9742 completed
1719/9742 completed
1720/9742 completed
1721/9742 completed
1722/9742 completed
1723/9742 completed
1724/9742 completed
1725/9742 completed
1726/9742 completed
1727/9742 completed
1728/9742 completed
1729/9742 completed
1730/9742 completed
1731/9742 completed
1732/9742 completed
1733/9742 completed
1734/9742 completed
1735/9742 completed
1736/9742 completed
1737/9742 completed
1738/9742 completed
1739/9742 completed
1740/9742 completed
1741/9742 completed
1742/9742 completed
1743/9742 completed
1744/9742 completed
1745/9742 completed
1746/9742 completed


...

8527/9742 completed
8528/9742 completed
8529/9742 completed
8530/9742 completed
8531/9742 completed
8532/9742 completed
8533/9742 completed
8534/9742 completed
8535/9742 completed
8536/9742 completed
8537/9742 completed
8538/9742 completed
8539/9742 completed
8540/9742 completed
8541/9742 completed
8542/9742 completed
8543/9742 completed
8544/9742 completed
8545/9742 completed
8546/9742 completed
8547/9742 completed
8548/9742 completed
8549/9742 completed
8550/9742 completed
8551/9742 completed
8552/9742 completed
8553/9742 completed
8554/9742 completed
8555/9742 completed
8556/9742 completed
8557/9742 completed
8558/9742 completed
8559/9742 completed
8560/9742 completed
8561/9742 completed
8562/9742 completed
8563/9742 completed
8564/9742 completed
8565/9742 completed
8566/9742 completed
8567/9742 completed
8568/9742 completed
8569/9742 completed
8570/9742 completed
8571/9742 completed
8572/9742 completed
8573/9742 completed
8574/9742 completed
8575/9742 completed
8576/9742 completed


8937/9742 completed
8938/9742 completed
8939/9742 completed
8940/9742 completed
8941/9742 completed
8942/9742 completed
8943/9742 completed
8944/9742 completed
8945/9742 completed
8946/9742 completed
8947/9742 completed
8948/9742 completed
8949/9742 completed
8950/9742 completed
8951/9742 completed
8952/9742 completed
8953/9742 completed
8954/9742 completed
8955/9742 completed
8956/9742 completed
8957/9742 completed
8958/9742 completed
8959/9742 completed
8960/9742 completed
8961/9742 completed
8962/9742 completed
8963/9742 completed
8964/9742 completed
8965/9742 completed
8966/9742 completed
8967/9742 completed
8968/9742 completed
8969/9742 completed
8970/9742 completed
8971/9742 completed
8972/9742 completed
8973/9742 completed
8974/9742 completed
8975/9742 completed
8976/9742 completed
8977/9742 completed
8978/9742 completed
8979/9742 completed
8980/9742 completed
8981/9742 completed
8982/9742 completed
8983/9742 completed
8984/9742 completed
8985/9742 completed
8986/9742 completed


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


9096/9742 completed
9097/9742 completed
9098/9742 completed
9099/9742 completed
9100/9742 completed
9101/9742 completed
9102/9742 completed
9103/9742 completed
9104/9742 completed
9105/9742 completed
9106/9742 completed
9107/9742 completed
9108/9742 completed
9109/9742 completed
9110/9742 completed
9111/9742 completed
9112/9742 completed
9113/9742 completed
9114/9742 completed
9115/9742 completed
9116/9742 completed
9117/9742 completed
9118/9742 completed
9119/9742 completed
9120/9742 completed
9121/9742 completed
9122/9742 completed
9123/9742 completed
9124/9742 completed
9125/9742 completed
9126/9742 completed
9127/9742 completed
9128/9742 completed
9129/9742 completed
9130/9742 completed
9131/9742 completed
9132/9742 completed
9133/9742 completed
9134/9742 completed
9135/9742 completed
9136/9742 completed
9137/9742 completed
9138/9742 completed
9139/9742 completed
9140/9742 completed
9141/9742 completed
9142/9742 completed
9143/9742 completed
9144/9742 completed
9145/9742 completed


9506/9742 completed
9507/9742 completed
9508/9742 completed
9509/9742 completed
9510/9742 completed
9511/9742 completed
9512/9742 completed
9513/9742 completed
9514/9742 completed
9515/9742 completed
9516/9742 completed
9517/9742 completed
9518/9742 completed
9519/9742 completed
9520/9742 completed
9521/9742 completed
9522/9742 completed
9523/9742 completed
9524/9742 completed
9525/9742 completed
9526/9742 completed
9527/9742 completed
9528/9742 completed
9529/9742 completed
9530/9742 completed
9531/9742 completed
9532/9742 completed
9533/9742 completed
9534/9742 completed
9535/9742 completed
9536/9742 completed
9537/9742 completed
9538/9742 completed
9539/9742 completed
9540/9742 completed
9541/9742 completed
9542/9742 completed
9543/9742 completed
9544/9742 completed
9545/9742 completed
9546/9742 completed
9547/9742 completed
9548/9742 completed
9549/9742 completed
9550/9742 completed
9551/9742 completed
9552/9742 completed
9553/9742 completed
9554/9742 completed
9555/9742 completed


In [ ]:
link = "https://www.themoviedb.org/movie/" + df_links["tmdbId"][0]

In [ ]:
bs = BeautifulSoup(r.content, "html.parser")

In [ ]:
print (bs.prettify)

In [ ]:
print(bs.p.get_text())